<a href="https://colab.research.google.com/github/coraldx5/generativeai_intro_book/blob/master/chap05_LLM_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLM（Masked Language Model）

In [2]:
!pip install fugashi ipadic
import transformers
from transformers import BertJapaneseTokenizer, BertForMaskedLM, pipeline

# トークナイザと訓練済みモデルの読み込み
# 'cl-tohoku/bert-base-japanese-whole-word-masking' という事前学習済みの日本語BERTモデルを使用します。
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# 事前学習済みモデルに対応するトークナイザーをロードします。
# BertJapaneseTokenizerはテキストをトークン（モデルが理解できる単位）に変換し、逆にトークンからテキストに変換する役割を持ちます。
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# パイプラインの定義
# 'fill-mask' タスクのパイプラインを作成します。これは文章の中の [MASK] トークンを予測するためのものです。
fill_mask = pipeline('fill-mask',
                     model=model,        # 言語モデルの指定
                     tokenizer=tokenizer, # トークナイザの指定
                     top_k=6              # 表示する候補数の指定
                    )

# 結果を表示する関数の定義
# 文章を入力として、[MASK] トークンの候補とその確率を表示する関数を定義します。
def predictmask(text):
    print('---' * 10)
    print(f'元の文章：「{text}」')
    print(f'[MASK]部の候補：')
    for res in fill_mask(text):
        print(f"{res['score']:.4f}: {res['token_str']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

In [3]:
predictmask('サングラスをかけた[MASK]が公園を駆け回る。')
predictmask('サングラスをかけた[MASK]を食べるのが楽しみだ')
predictmask('生卵をかけた[MASK]を食べるのが楽しみだ')

------------------------------
元の文章：「サングラスをかけた[MASK]が公園を駆け回る。」
[MASK]部の候補：
0.2395: 男
0.0848: 少年
0.0511: 少女
0.0433: 犬
0.0331: 青年
0.0293: 女性
------------------------------
元の文章：「サングラスをかけた[MASK]を食べるのが楽しみだ」
[MASK]部の候補：
0.0882: もの
0.0608: ケーキ
0.0493: 料理
0.0314: 朝食
0.0312: 犬
0.0305: ご飯
------------------------------
元の文章：「生卵をかけた[MASK]を食べるのが楽しみだ」
[MASK]部の候補：
0.1659: もの
0.1010: ご飯
0.0815: 料理
0.0752: スープ
0.0658: 卵
0.0490: パン


In [4]:
# @title ## UIの参考：穴埋め問題を解くサンプルコード
# @markdown 入力欄に、[MASK]を含む文章を入力してください。
check_sentence = "サングラスをかけた[MASK]が公園を駆け回る。" # @param {type:"string"}
predictmask(check_sentence)

------------------------------
元の文章：「サングラスをかけた[MASK]が公園を駆け回る。」
[MASK]部の候補：
0.2395: 男
0.0848: 少年
0.0511: 少女
0.0433: 犬
0.0331: 青年
0.0293: 女性


# CLM（Causal Language Model）

In [ ]:
import transformers
from transformers import GPT2LMHeadModel, T5Tokenizer

# モデルとトークナイザーのロード
# 'rinna/japanese-gpt2-medium' という事前学習済みの日本語GPT-2モデルを使用します。モデルのサイズは約1.37GBです。
model_name = 'rinna/japanese-gpt2-medium'  # モデルの名前を指定

# 事前学習済みのGPT-2モデルをロードします。GPT2LMHeadModelはテキスト生成タスクに使用されるモデルです。
model = GPT2LMHeadModel.from_pretrained(model_name)

# 事前学習済みモデルに対応するトークナイザーをロードします。
# T5Tokenizerはテキストをトークン（モデルが理解できる単位）に変換し、逆にトークンからテキストに変換する役割を持ちます。
tokenizer = T5Tokenizer.from_pretrained(model_name)

def generate_text(input_text, max_length):
    # 入力文章をトークン化
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Attention maskの設定
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

    # テキスト生成
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,  # Attention maskを指定
        max_length=max_length,  # 生成する最大トークン数
        # no_repeat_ngram_size=2,  # 繰り返しを防ぐn-gramのサイズ
        pad_token_id=tokenizer.pad_token_id,  # パディングのトークンID
        bos_token_id=tokenizer.bos_token_id,  # テキスト先頭のトークンID
        eos_token_id=tokenizer.eos_token_id,  # テキスト終端のトークンID
    )

    # 生成されたテキストのデコード
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text

In [ ]:
# 関数の使用例
input_text = "これから雨が降りそうなので、"  # 入力テキスト
max_length = 50  # 生成する最大トークン数

# 生成されたテキストを表示
generated_text = generate_text(input_text, max_length)
print(generated_text)

これから雨が降りそうなので、今日は、お休みです。 明日は、お休みです。 今日は、お休みです。 今日は、お休みです。 今日は、お休みです。 今日


In [ ]:
# @title ## UIの参考：後続文章を生成するサンプルコード
# @markdown 文章を入力して下さい
input_text = "これから雨が降りそうなので、" # @param {type:"string"}
# @markdown 生成する最大トークン数
max_length = 50 # @param {type:"integer"}
generate_text(input_text, max_length)

'これから雨が降りそうなので、今日は、お休みです。 明日は、お休みです。 今日は、お休みです。 今日は、お休みです。 今日は、お休みです。 今日'

## 各トークンの予測確率を可視化してみよう

In [ ]:
# モデルを評価モードに設定
model.eval()

# 入力文章をトークン化
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Attention maskの設定
attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

# 生成されたトークンとその確率を順次表示
max_length = 10  # 生成する最大トークン数
for _ in range(max_length):
    # トークンの予測確率を取得
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits

    # 次のトークンの予測確率を計算
    next_token_probs = torch.softmax(predictions[:, -1, :], dim=-1)

    # 上位3つのトークンを取得
    top_k = 3
    top_k_probs, top_k_indices = torch.topk(next_token_probs, top_k)

    # 上位3つのトークンとその確率を表示
    print(f"\n({_+1}番目) 上位3つのトークンと、確率：")
    for i in range(top_k):
        predicted_token_id = top_k_indices[0, i].item()
        predicted_token = tokenizer.decode([predicted_token_id])
        predicted_prob = top_k_probs[0, i].item()
        print(f"Token: {predicted_token}({predicted_token_id}), Probability: {predicted_prob:.4f}")

    # 最も確率の高いトークンを入力トークンに追加
    input_ids = torch.cat((input_ids, top_k_indices[:, 0].unsqueeze(-1)), dim=1)

    # 予測が終了トークンに到達した場合は終了
    if top_k_indices[0, 0].item() == tokenizer.eos_token_id:
        break

# 生成された全テキストを表示
generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print("---"*10)
print("生成された文章:")
print(generated_text)


(1番目) 上位3つのトークンと、確率：
Token: 今日(4761), Probability: 0.0668
Token: 雨(1537), Probability: 0.0178
Token: 早(1745), Probability: 0.0128

(2番目) 上位3つのトークンと、確率：
Token: は(11), Probability: 0.7620
Token: も(30), Probability: 0.0637
Token: はこの(1007), Probability: 0.0170

(3番目) 上位3つのトークンと、確率：
Token: 、(7), Probability: 0.1085
Token: (9), Probability: 0.0632
Token: お(220), Probability: 0.0614

(4番目) 上位3つのトークンと、確率：
Token: お(220), Probability: 0.0566
Token: (9), Probability: 0.0373
Token: いつも(9328), Probability: 0.0168

(5番目) 上位3つのトークンと、確率：
Token: 休み(19138), Probability: 0.1112
Token: 昼(5766), Probability: 0.0946
Token: 散歩(22925), Probability: 0.0602

(6番目) 上位3つのトークンと、確率：
Token: です(2767), Probability: 0.1763
Token: を(18), Probability: 0.1137
Token: に(17), Probability: 0.0931

(7番目) 上位3つのトークンと、確率：
Token: 。(8), Probability: 0.6789
Token: (9), Probability: 0.0664
Token: ((15), Probability: 0.0511

(8番目) 上位3つのトークンと、確率：
Token: (9), Probability: 0.6433
Token: お(14287), Probability: 0.0382
Token: 。(8), Probab

In [ ]:
import torch
from transformers import GPT2LMHeadModel, T5Tokenizer

# モデルとトークナイザーのロード
model_name = 'rinna/japanese-gpt2-medium'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# テキスト生成の設定
input_text = "これから雨が降りそうなので、"  # 入力テキスト
max_length = 40  # 生成する最大トークン数

# トークン化
# 入力テキストをモデルが理解できるトークンに変換します
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# テキスト生成のパラメータを辞書で設定
# max_length: 生成する最大トークン数
# pad_token_id: パディングのトークンID
# bos_token_id: テキスト先頭のトークンID
# eos_token_id: テキスト終端のトークンID
prm = {
    "max_length": max_length,
    "pad_token_id": tokenizer.pad_token_id,
    "bos_token_id": tokenizer.bos_token_id,
    "eos_token_id": tokenizer.eos_token_id,
}

# Greedy探索
# Greedy探索は、各ステップで最も確率の高いトークンを選びます
greedy_output = model.generate(input_ids, **prm)
print("Greedy:", tokenizer.decode(greedy_output[0], skip_special_tokens=True))

# Beam探索
# Beam探索は、複数の候補（ビーム）を同時に探索し、最も良い結果を選びます
# num_beams: ビームの数
# early_stopping: 生成の早期終了を行うかどうか
beam_output = model.generate(input_ids, num_beams=3, early_stopping=True, **prm)
print("Beam:", tokenizer.decode(beam_output[0], skip_special_tokens=True))

# Top-kサンプリング
# Top-kサンプリングは、上位k個のトークンからランダムに選択します
# do_sample: サンプリングを行うかどうか
# top_k: 選択する上位トークンの数
top_k_output = model.generate(input_ids, do_sample=True, top_k=50, **prm)
print("Top-k Sampling:", tokenizer.decode(top_k_output[0], skip_special_tokens=True))

# Top-pサンプリング
# Top-pサンプリングは、確率の高いトークンの集合からランダムに選択します
# do_sample: サンプリングを行うかどうか
# top_p: 累積確率がpを超えるまでのトークンを選択する閾値
top_p_output = model.generate(input_ids, do_sample=True, top_p=0.95, **prm)
print("Top-p Sampling:", tokenizer.decode(top_p_output[0], skip_special_tokens=True))


Greedy: これから雨が降りそうなので、今日は、お休みです。 明日は、お休みです。 今日は、お休みです。 今日は、お休みです。
Beam: これから雨が降りそうなので、早めに切り上げました。 今日は、朝から雨が降っていましたが、 午後からは晴れてきました。 今日は
Top-k Sampling: これから雨が降りそうなので、近所の公園まで自転車を走らせることに。 先日お弁当食べてたときに、このことを言われました。 いやぁ～
Top-p Sampling: これから雨が降りそうなので、自転車に乗っている方には 傘をお願いします。 昨日は、お見舞いの品を届けに行ってきました。。。


In [ ]:
# @title ## UIの参考：様々な探索法で文章を生成するサンプルコード
# @markdown 文章を入力して下さい
# テキスト生成の設定
input_text = "これから雨が降りそうなので、" # @param {type:"string"}
# @markdown 生成する最大トークン数
max_length = 50 # @param {type:"integer"}

# トークン化
# 入力テキストをモデルが理解できるトークンに変換します
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# テキスト生成のパラメータを辞書で設定
# max_length: 生成する最大トークン数
# pad_token_id: パディングのトークンID
# bos_token_id: テキスト先頭のトークンID
# eos_token_id: テキスト終端のトークンID
prm = {
    "max_length": max_length,
    "pad_token_id": tokenizer.pad_token_id,
    "bos_token_id": tokenizer.bos_token_id,
    "eos_token_id": tokenizer.eos_token_id,
}

# Greedy探索
# Greedy探索は、各ステップで最も確率の高いトークンを選びます
greedy_output = model.generate(input_ids, **prm)
print("Greedy:", tokenizer.decode(greedy_output[0], skip_special_tokens=True))

# Beam探索
# Beam探索は、複数の候補（ビーム）を同時に探索し、最も良い結果を選びます
# num_beams: ビームの数
# early_stopping: 生成の早期終了を行うかどうか
beam_output = model.generate(input_ids, num_beams=3, early_stopping=True, **prm)
print("Beam:", tokenizer.decode(beam_output[0], skip_special_tokens=True))

# Top-kサンプリング
# Top-kサンプリングは、上位k個のトークンからランダムに選択します
# do_sample: サンプリングを行うかどうか
# top_k: 選択する上位トークンの数
top_k_output = model.generate(input_ids, do_sample=True, top_k=50, **prm)
print("Top-k Sampling:", tokenizer.decode(top_k_output[0], skip_special_tokens=True))

# Top-pサンプリング
# Top-pサンプリングは、確率の高いトークンの集合からランダムに選択します
# do_sample: サンプリングを行うかどうか
# top_p: 累積確率がpを超えるまでのトークンを選択する閾値
top_p_output = model.generate(input_ids, do_sample=True, top_p=0.95, **prm)
print("Top-p Sampling:", tokenizer.decode(top_p_output[0], skip_special_tokens=True))

# worv2vec
- [東北大学のページ](https://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/)から学習済モデル(20170201.tar.bz2)をダウンロードして使用します。
- 同ホームページからの抜粋：
  - 「日本語 Wikipedia エンティティベクトル」は、[日本語版 Wikipedia](https://ja.wikipedia.org/wiki/) の本文全文から学習した、単語、および Wikipedia で記事となっているエンティティの分散表現ベクトルです。Wikipedia の記事本文の抽出には [WikiExtractor ](https://github.com/attardi/wikiextractor)を、単語分割には [MeCab](http://taku910.github.io/mecab/) を、単語ベクトルの学習には [word2vec](https://code.google.com/archive/p/word2vec/
) をそれぞれ用いています。

In [ ]:
!wget https://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2
!tar xf 20170201.tar.bz2

--2024-05-27 14:56:50--  https://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2
Resolving www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1373795477 (1.3G) [application/x-bzip2]
Saving to: ‘20170201.tar.bz2’

20170201.tar.bz2    100%[===================>]   1.28G  11.2MB/s    in 2m 8s   

2024-05-27 14:58:59 (10.2 MB/s) - ‘20170201.tar.bz2’ saved [1373795477/1373795477]



output_file は結果を書き込むファイルを開くための変数です。result.txt という名前のファイルに書き込みます。
display_and_save_list 関数は、リスト内の単語とその類似度を表示し、ファイルに書き込むための関数です。
word2vec_model は、事前に訓練されたWord2Vecモデルをロードするための変数です。
similar_words_mickey は、ミッキーと類似度が高い単語のリストを取得します。
similar_words_mickey_christmas は、ミッキーとクリスマスの両方に関連する単語のリストを取得します。
similar_words_usj は、ミッキーからディズニーを引き、ユニバーサルスタジオジャパンを足した場合に関連する単語のリストを取得します。

In [ ]:
import codecs
from gensim.models import KeyedVectors

# 結果を保存するファイルを開く
output_file = codecs.open('result.txt', 'w', 'utf-8')

# リストを表示およびファイルに書き込む関数
def display_and_save_list(word_list):
    for word, similarity in word_list:
        # 表示
        print(word, str(similarity))
        # ファイルに書き込み
        output_file.write(f"{word},{similarity}\n")
    print("---")

# モデルをロード
word2vec_model = KeyedVectors.load_word2vec_format('./entity_vector/entity_vector.model.bin', binary=True)

# 王様と類似度が高い言葉を取得
similar_words_mickey = word2vec_model.most_similar('王様')
display_and_save_list(similar_words_mickey)

# 王様とブルボン朝と類似の言葉を取得（王様 + ブルボン朝）
similar_words_mickey_christmas = word2vec_model.most_similar(positive=['王様', 'ブルボン朝'])
display_and_save_list(similar_words_mickey_christmas)

# ユニバーサルスタジオ界のミッキーは誰か（ミッキー - ディズニー + ユニバーサルスタジオジャパン）
similar_words_usj = word2vec_model.most_similar(positive=['王様', '江戸時代'], negative=['ブルボン朝'])
display_and_save_list(similar_words_usj)


お姫様 0.7911006808280945
おじいさん 0.708034098148346
[シンデレラ] 0.7062105536460876
[アリス_(不思議の国のアリス)] 0.7054476737976074
魔法使い 0.6919453740119934
妖精 0.6900907754898071
貴婦人 0.6897401809692383
パパ 0.6879994869232178
[白雪姫] 0.6826063990592957
魔女 0.6819620132446289
---
エツィオ 0.6156821250915527
ディド 0.6051962971687317
シッド 0.6026549339294434
[フロージ] 0.5943021178245544
王笏 0.5941326022148132
エイゴン 0.5928665995597839
[ロスタム] 0.5919443368911743
[ウォート] 0.5914792418479919
ハーコン 0.5900202393531799
リヴァ 0.588994562625885
---
殿様 0.6322947144508362
芸者 0.5856623649597168
神様 0.5821214318275452
老婆 0.5772008895874023
遊女 0.5680871605873108
狸 0.5647869110107422
[遊女] 0.5636616945266724
御馳走 0.5611592531204224
[花魁] 0.560475766658783
女房 0.5492725372314453
---


In [ ]:
# モデルのパラメータ数を確認
vocab_size = len(word2vec_model.key_to_index)  # 語彙サイズ
vector_size = word2vec_model.vector_size       # 各ベクトルの次元数
total_parameters = vocab_size * vector_size    # パラメータの総数

print(f"語彙サイズ: {vocab_size}")
print(f"ベクトル次元数: {vector_size}")
print(f"パラメータの総数: {total_parameters}")


語彙サイズ: 1015474
ベクトル次元数: 200
パラメータの総数: 203094800
